In [25]:
from final_data import get_final_data
from functions import percentage_within_threshold

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = get_final_data()

/Users/jennifermei/Documents/GitHub/PIC16B_Politics/final_data.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data = df_data[~remove_ca44]


In [43]:
data = data.dropna(axis=0)
data.head()

,Year,State,State Abbr,State FIPS,District,Democratic %,Republican %,Voter Turnout %,Median Age,Median income,...,Less than HS %,HS grad %,Some college %,Bachelor's degree %,Graduate degree %,Below poverty line %,Income under 25k %,Income 25k to 50k %,Income 50k to 75k %,Income above 75k %
0,2014,Alabama,AL,1,1,31.71,68.16,28.48,38.7,23092,...,9.74,21.99,20.23,9.87,5.09,10.38,60.15,22.48,9.76,7.61
1,2014,Alabama,AL,1,2,32.56,67.34,31.59,37.8,22525,...,11.34,21.55,19.61,8.86,5.36,10.34,59.82,23.11,9.69,7.38
2,2014,Alabama,AL,1,3,33.72,66.12,28.89,37.7,21508,...,11.87,21.44,19.35,8.00,5.23,14.25,61.61,22.06,9.55,6.77
3,2014,Alabama,AL,1,4,0.00,98.57,25.38,40.5,20768,...,14.21,23.01,20.46,6.54,4.07,18.83,63.63,22.21,8.59,5.57
4,2014,Alabama,AL,1,5,0.00,74.42,28.23,38.8,25783,...,9.65,18.43,19.82,12.44,7.32,11.20,56.04,21.81,10.48,11.67


In [45]:
# features for training
train_columns = ['Year', 'State FIPS', 'District', 'Voter Turnout %', 'Median Age', 'Median income',
                 'Male Population %', 'Female Population %', 'White %', 'White (non Hispanic or Latino) %',
                 'Hispanic or Latino %', 'Black %', 'American Indian %', 'Asian %', 'Pacific %', 'Multiracial %',
                 'Less than HS %', 'HS grad %', 'Some college %', "Bachelor's degree %", "Graduate degree %",
                 'Below poverty line %', 'Income under 25k %', 'Income 25k to 50k %', 'Income 50k to 75k %',
                 'Income above 75k %']

In [46]:
# Select features for training
X = data[data['Year'] != 2020][train_columns].values

# Select target variables
y_democratic = data[data['Year'] != 2020]['Democratic %'].values
y_republican = data[data['Year'] != 2020]['Republican %'].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [47]:
# Train-test split for model training (use 80% for training, 20% for validation)
X_train, X_val, y_train_democratic, y_val_democratic, y_train_republican, y_val_republican = train_test_split(
    X, y_democratic, y_republican, test_size=0.2, random_state=42
)

In [48]:
# Define and train the Random Forest Regressor for both Democratic % and Republican %
rf_democratic = RandomForestRegressor(random_state=42)
rf_democratic.fit(X_train, y_train_democratic)

rf_republican = RandomForestRegressor(random_state=42)
rf_republican.fit(X_train, y_train_republican)

# Predict on the validation set for both Democratic % and Republican %
predictions_democratic_rf = rf_democratic.predict(X_val)
predictions_republican_rf = rf_republican.predict(X_val)

predictions_democratic_rf = np.clip(predictions_democratic_rf, 0, 100)
predictions_republican_rf = np.clip(predictions_republican_rf, 0, 100)

# Evaluate the model performance on the validation set for Democratic % using Mean Squared Error
mse_democratic = mean_squared_error(y_val_democratic, predictions_democratic_rf)
mse_republican = mean_squared_error(y_val_republican, predictions_republican_rf)

# Calculate the percentage within 5% and 10% threshold for Democratic %
accuracy_within_5_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_rf, threshold_percent=5)
accuracy_within_10_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_rf, threshold_percent=10)
accuracy_within_5_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_rf, threshold_percent=5)
accuracy_within_10_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_rf, threshold_percent=10)

print("RF Mean Squared Error for Democratic % on Validation Set:", mse_democratic)
print("RF Mean Squared Error for Republican % on Validation Set:", mse_republican)
print(f"RF Percentage of predictions within 5% for Democratic %: {accuracy_within_5_percent_democratic:.2f}%")
print(f"RF Percentage of predictions within 10% for Democratic %: {accuracy_within_10_percent_democratic:.2f}%")
print(f"RF Percentage of predictions within 5% for Republican %: {accuracy_within_5_percent_republican:.2f}%")
print(f"RF Percentage of predictions within 10% for Republican %: {accuracy_within_10_percent_republican:.2f}%")

print("Predicted Democratic % values:")
print(predictions_democratic_rf)
print("\nPredicted Republican % values:")
print(predictions_republican_rf)

RF Mean Squared Error for Democratic % on Validation Set: 247.1330302277992
RF Mean Squared Error for Republican % on Validation Set: 231.2745419953668
RF Percentage of predictions within 5% for Democratic %: 20.08%
RF Percentage of predictions within 10% for Democratic %: 40.93%
RF Percentage of predictions within 5% for Republican %: 25.10%
RF Percentage of predictions within 10% for Republican %: 42.86%
Predicted Democratic % values:
[ 45.45    51.2073  32.4079  38.9388  40.4224  74.6183  49.6361  39.3829
  61.0392  24.2043  74.3632  33.0481  76.3472 100.      30.6716  44.9836
  47.0382  62.865   44.7974  49.6645  16.5759  60.6146  46.3794  51.9216
  48.1248  39.7498  17.0541  64.8398  41.8357  46.6847  60.283   78.6179
  39.306   54.2708  70.0735  57.7882  67.8091  38.7576  33.7268  25.0453
  44.5276  63.12    58.2984  49.7497  30.5071 100.      24.2775  69.8664
  49.963   34.729   66.9272  43.9877  39.2189  72.7624  60.0287 100.
  27.6652  62.8434  39.4855  74.0074  42.4047  33.00

In [49]:
# Define and train the Linear Regression model for both Democratic % and Republican %
lr_democratic = LinearRegression()
lr_democratic.fit(X_train, y_train_democratic)

lr_republican = LinearRegression()
lr_republican.fit(X_train, y_train_republican)

# Predict on the validation set for both Democratic % and Republican %
predictions_democratic_lr = lr_democratic.predict(X_val)
predictions_republican_lr = lr_republican.predict(X_val)

predictions_democratic_lr = np.clip(predictions_democratic_lr, 0, 100)
predictions_republican_lr = np.clip(predictions_republican_lr, 0, 100)

# Evaluate the model performance on the validation set for Democratic % using Mean Squared Error
mse_democratic = mean_squared_error(y_val_democratic, predictions_democratic_lr)
mse_republican = mean_squared_error(y_val_republican, predictions_republican_lr)

# Calculate the percentage within 5% and 10% threshold for Democratic %
accuracy_within_5_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_rf, threshold_percent=5)
accuracy_within_10_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_rf, threshold_percent=10)
accuracy_within_5_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_lr, threshold_percent=5)
accuracy_within_10_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_lr, threshold_percent=10)

print("LR Mean Squared Error for Democratic % on Validation Set:", mse_democratic)
print("LR Mean Squared Error for Republican % on Validation Set:", mse_republican)
print(f"LR Percentage of predictions within 5% for Democratic %: {accuracy_within_5_percent_democratic:.2f}%")
print(f"LR Percentage of predictions within 10% for Democratic %: {accuracy_within_10_percent_democratic:.2f}%")
print(f"LR Percentage of predictions within 5% for Republican %: {accuracy_within_5_percent_republican:.2f}%")
print(f"LR Percentage of predictions within 10% for Republican %: {accuracy_within_10_percent_republican:.2f}%")

print("Predicted Democratic % values:")
print(predictions_democratic_lr)
print("\nPredicted Republican % values:")
print(predictions_republican_lr)

LR Mean Squared Error for Democratic % on Validation Set: 372.97121910310517
LR Mean Squared Error for Republican % on Validation Set: 368.0229674466167
LR Percentage of predictions within 5% for Democratic %: 20.08%
LR Percentage of predictions within 10% for Democratic %: 40.93%
LR Percentage of predictions within 5% for Republican %: 17.76%
LR Percentage of predictions within 10% for Republican %: 29.73%
Predicted Democratic % values:
[ 51.86984374  54.62759579  47.99599559  56.20818496  36.86524929
  96.49882163  68.66314629  52.08423749  73.97981495  11.09314577
  86.89008381  36.38373553  39.33488011  57.4216586   44.81912172
  35.79774369  55.68578122  61.72381269  54.57852014  54.10975944
  37.50315954  49.83400631  40.02647492  59.38739893  25.76141028
  54.10106278  25.18176692  77.7094176   57.12388234  42.66363536
  88.91218581  64.82012561  40.45962175  47.5942703   46.53866537
  66.01446672  59.40751562  25.90421378  38.6356596   31.01234963
  29.41684303  82.9798847   92

In [50]:
# Define and train the Gradient Boosting Regressor model for both Democratic % and Republican %
gb_democratic = GradientBoostingRegressor(random_state=42)
gb_democratic.fit(X_train, y_train_democratic)

gb_republican = GradientBoostingRegressor(random_state=42)
gb_republican.fit(X_train, y_train_republican)

# Predict on the validation set for both Democratic % and Republican %
predictions_democratic_gb = gb_democratic.predict(X_val)
predictions_republican_gb = gb_republican.predict(X_val)

predictions_democratic_gb = np.clip(predictions_democratic_gb, 0, 100)
predictions_republican_gb = np.clip(predictions_republican_gb, 0, 100)

# Evaluate the model performance on the validation set for Democratic % using Mean Squared Error
mse_democratic = mean_squared_error(y_val_democratic, predictions_democratic_gb)
mse_republican = mean_squared_error(y_val_republican, predictions_republican_gb)

# Calculate the percentage within 5% and 10% threshold for Democratic %
accuracy_within_5_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_gb, threshold_percent=5)
accuracy_within_10_percent_democratic = percentage_within_threshold(y_val_democratic, predictions_democratic_gb, threshold_percent=10)
accuracy_within_5_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_gb, threshold_percent=5)
accuracy_within_10_percent_republican = percentage_within_threshold(y_val_republican, predictions_republican_gb, threshold_percent=10)

print("GB Mean Squared Error for Democratic % on Validation Set:", mse_democratic)
print("GB Mean Squared Error for Republican % on Validation Set:", mse_republican)
print(f"GB Percentage of predictions within 5% for Democratic %: {accuracy_within_5_percent_democratic:.2f}%")
print(f"GB Percentage of predictions within 10% for Democratic %: {accuracy_within_10_percent_democratic:.2f}%")
print(f"GB Percentage of predictions within 5% for Republican %: {accuracy_within_5_percent_republican:.2f}%")
print(f"GB Percentage of predictions within 10% for Republican %: {accuracy_within_10_percent_republican:.2f}%")

print("Predicted Democratic % values:")
print(predictions_democratic_gb)
print("\nPredicted Republican % values:")
print(predictions_republican_gb)

GB Mean Squared Error for Democratic % on Validation Set: 218.68812287981285
GB Mean Squared Error for Republican % on Validation Set: 283.4572613769281
GB Percentage of predictions within 5% for Democratic %: 20.46%
GB Percentage of predictions within 10% for Democratic %: 36.29%
GB Percentage of predictions within 5% for Republican %: 14.67%
GB Percentage of predictions within 10% for Republican %: 33.98%
Predicted Democratic % values:
[ 51.83814409  50.93330993  31.90980642  32.63151176  45.26347034
  67.99628869  51.02550463  39.70788313  71.93662382  22.25259069
  73.592196    31.86733181  77.10511257 100.          25.14978896
  36.5302977   45.11381156  46.83826866  48.95688102  46.31814216
  32.2471997   55.61597463  41.49685209  51.6533163   45.61394586
  33.00989649  25.28403083  66.93545509  58.18407271  55.60798979
  61.38346276  86.80792167  30.9317297   39.34841876  44.29828321
  52.05663443  64.1131796   35.43569176  33.58308016  27.35540077
  42.19881873  55.15376871  67